# MLP 

In [1]:
from datetime import datetime
print(datetime.now())
from platform import python_version
print('Python',python_version())
from csv import reader
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import math
import random
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

2022-10-11 04:17:06.219023
Python 3.10.0
sklearn 1.1.2


In [2]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import LayerNormalization

dt='float32'
tf.keras.backend.set_floatx(dt)

from KmerCounter import KmerCounter
K=4
counter = KmerCounter()
counter.setK(K)
VOCABULARY_SIZE = counter.get_vocabulary_size()

from TrainValidSplit import Splitter
EPOCHS=10
SPLITS=3
EMBED_DIMEN=16

from cell_lines import Cell_Lines

In [3]:
ATLAS_DIR = '/Users/jasonmiller/WVU/Localization/LncAtlas/'
RCI_FILE = 'CNRCI_coding_train_genes.csv'
GENCODE_DIR = '/Users/jasonmiller/WVU/Localization/GenCode/'
SEQUENCE_FILE = 'Homo_sapiens.GRCh38.cds.csv'

In [4]:
all_cell_lines = Cell_Lines.get_ordered_list()
cell_line_number = 0
cell_line_name = all_cell_lines[cell_line_number]
print('Cell line for today:',cell_line_number,cell_line_name)

Cell line for today: 0 A549


## Load labels and sequences

In [5]:
def load_labels(atlas_file,cells):
    gene_labels = {}
    with open(atlas_file,'r') as atlas:
        header = None
        genes_considered = 0
        positives = 0
        csv = reader(atlas)
        for row in csv:
            if header is None:
                header = row
            else:
                gene = row[0]
                genes_considered += 1
                rci = float(row[1+cells])
                if not math.isnan(rci):
                    # GENERATE BINARY LABELS
                    label = 0   # RCI<0
                    if rci>=0:  # THRESHOLD
                        label = 1   #RCI>=0
                        positives += 1
                    gene_labels[gene]=label
    print('Genes:',genes_considered)
    print('Gene labels:',len(gene_labels))
    print('Positive labels:',positives)
    return gene_labels
    
print(datetime.now())
atlas_path = ATLAS_DIR+RCI_FILE
gene_label = load_labels(atlas_path,cell_line_number)

2022-10-11 04:17:18.321603
Genes: 14216
Gene labels: 10392
Positive labels: 5412


In [6]:
def load_sequences(gencode_file,gene_label):
    gid_tid = []
    labels = []
    tseqs = []
    with open(gencode_file,'r') as gencode:
        header = None
        csv = reader(gencode)
        for row in csv:
            if header is None:
                header = row
            else:
                tran_id = row[0]
                gene_id = row[1]
                tseq = row[4]
                if gene_id in gene_label:
                    label = gene_label[gene_id]
                    gid_tid.append ((gene_id,tran_id))
                    labels.append(label)
                    tseqs.append(tseq)
    return gid_tid,labels,tseqs

In [7]:
print(datetime.now())
gencode_path = GENCODE_DIR+SEQUENCE_FILE
ordered_gid_tid,ordered_labels,ordered_seqs = \
    load_sequences(gencode_path,gene_label)

2022-10-11 04:17:18.428533


## Make K-mers

In [9]:
print(datetime.now())
ordered_kmer_counts = []
tock = 1000
tick = 0
for seq in ordered_seqs:
    tick += 1
    if tick >= tock:
        tick = 1
        print('.',end='')
    counts = counter.seq_to_kmer_counts(seq)
    ordered_kmer_counts.append(counts)
print()
print(datetime.now())

2022-10-11 04:18:24.547609
.....................................................
2022-10-11 04:25:59.889751


In [14]:
splitter=Splitter(ordered_gid_tid,ordered_kmer_counts,ordered_labels)

## Build model

In [15]:
def build_model():
    act="sigmoid"
    #embed_layer  = keras.layers.Embedding(
    #    VOCABULARY_SIZE,EMBED_DIMEN,input_length=maxlen);
    dense1_layer = keras.layers.Dense(64, activation=act,dtype=dt,
                                      input_dim=VOCABULARY_SIZE)
    dense2_layer = keras.layers.Dense(64, activation=act,dtype=dt)
    dense3_layer = keras.layers.Dense(64, activation=act,dtype=dt)
    #output_layer = keras.layers.Dense(1,  activation="softmax",dtype=dt)
    output_layer = keras.layers.Dense(1,  activation=act,dtype=dt)

    mlp = keras.models.Sequential()
    #mlp.add(embed_layer)
    mlp.add(dense1_layer)
    mlp.add(dense2_layer)
    mlp.add(dense3_layer)
    mlp.add(output_layer)
    
    # Logit=True ranges from + to - infinity.
    # Logit=False i.e. probabilities range from 0 to 1.
    # If your output layer has a 'softmax' activation, from_logits should be False. If your output layer doesn't have a 'softmax' activation, from_logits should be True. 
    bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
    print("COMPILE...")
    mlp.compile(loss=bc, optimizer="Adam",metrics=["accuracy"])
    print("...COMPILED")
    return mlp

## Cross validation

In [16]:
def do_cross_validation(X,y,eps):
    cv_scores = []
    fold=0
    PARTITIONS=5
    # PARTITIONS=1   # for testing
    for i in range(PARTITIONS):
        X_train,y_train,X_valid,y_valid = \
            splitter.train_valid_split(i,PARTITIONS)
        print('Fold',i)
        print('Train set ones/size',
              np.count_nonzero(y_train),'/',len(y_train))
        print('Valid set ones/size',
              np.count_nonzero(y_valid),'/',len(y_valid))

        print("BUILD MODEL")
        model=build_model()

        print("FIT")
        # this is complaining about string to float
        history=model.fit(X_train, y_train, # batch_size=10, default=32 works nicely
                epochs=eps, verbose=1,  # verbose=1 for ascii art, verbose=0 for none
                validation_data=(X_valid,y_valid) )
                        
        fold += 1
        print("Fold %d, %d epochs"%(fold,eps))

        pd.DataFrame(history.history).plot(figsize=(8,5))
        plt.grid(True)
        plt.gca().set_ylim(0,1)
        plt.show()

        scores = model.evaluate(X_valid, y_valid, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        # What are the other metrics_names?
        # Try this from Geron page 505:
        # np.mean(keras.losses.mean_squared_error(y_valid,y_pred))
        cv_scores.append(scores[1] * 100)
    print()
    print("Validation core mean %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))


## Train

In [17]:
print(datetime.now())
model=build_model()
print ("Summarize the model")
print(model.summary())  # Print this only once

print ("Sequence to Kmer")
y = ordered_labels
X = ordered_kmer_counts

print ("Cross valiation")
do_cross_validation(X,y,EPOCHS)
print(datetime.now())

2022-10-11 04:30:51.110292
COMPILE...
...COMPILED
Summarize the model
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 24,833
Trainable params: 24,833
Non-trainable params: 0
_________________________________________________________________
None
Sequence to Kmer
Cross valiation


TypeError: Splitter.train_valid_split() takes 2 positional arguments but 3 were given

# to do
Do train/valid split with respect to genes. 